In [2]:
import tensorflow as tf
import numpy as np
import time
import math
import Cifar10_data

In [8]:
int(2.3)

2

In [1]:
max_steps = 4000
batch_size = 100
num_examples_for_eval = 10000 
data_dir = 'Cifar_data/cifar-10-batches-bin/'



In [10]:
# 创建一个函数，variable_with_weight_loss()
# 作用
# 1. 使用参数w1控制L2 loss的大小
# 2. 使用函数tf.nn.l2_loss()计算权重L2 loss
# 3. 使用tf.multiply()计算权重L2 loss 与w1的乘积， 并赋值给weights_loss
# 4. 使用函数tf.add_to_collection()将最终的结果放在名为losses的集合里面，方
# 便后面计算神经网络总体损失loss


def variable_with_weight_loss(shape, stddev, w1):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if w1 is not None:
        weights_loss = tf.multiply(tf.nn.l2_loss(var), w1, name="weights_loss")
        tf.add_to_collection('losses', weights_loss)
    return var


In [4]:
images_train, labels_train = Cifar10_data.inputs(data_dir=data_dir,batch_size=batch_size,
                                                distorted=True)
images_test, labels_test = Cifar10_data.inputs(data_dir=data_dir, batch_size=batch_size,
                                               distorted=False)


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.

Instructions for updating:


In [5]:
x = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
y = tf.placeholder(tf.int32, [batch_size])


In [43]:
# 创建第一个卷积层 shape=(kh, hw, ci, co)
k1 = tf.constant([[[2,3],[3,4]]])

s2 = k1.get_shape()  # k1.get_shape().as_list()
print('s2', s2)
kernel1 = variable_with_weight_loss(shape=[5, 5, 3, 64], stddev=5e-2, w1=0.0)
conv1 = tf.nn.conv2d(x, kernel1, [1,1,1,1], padding="SAME")  #  https://blog.csdn.net/theadore2017/article/details/107232208/
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(tf.shape(kernel1)))
    print(sess.run(tf.shape(kernel1)[0]))
    print(sess.run(tf.shape(conv1)))
    
    print(sess.run(tf.shape(k1)))
    

s2 (1, 2, 2)
[ 5  5  3 64]
5
[100  24  24  64]
[1 2 2]


In [ ]:
tf.nn.conv2d(
    input,   # 
    filter=None,
    strides=None,
    padding=None,
    use_cudnn_on_gpu=True,
    data_format='NHWC',
    dilations=[1, 1, 1, 1],
    name=None,
    filters=None,
)

Flattens the filter to a 2-D matrix with shape
   `[filter_height * filter_width * in_channels, output_channels]`.
2. Extracts image patches from the input tensor to form a *virtual*
   tensor of shape `[batch, out_height, out_width,
   filter_height * filter_width * in_channels]`.
3. For each patch, right-multiplies the filter matrix and the image patch
   vector.

In detail, with the default NHWC format,

    output[b, i, j, k] =
        sum_{di, dj, q} input[b, strides[1] * i + di, strides[2] * j + dj, q]
                        * filter[di, dj, q, k]

Must have `strides[0] = strides[3] = 1`.  For the most common case of the same
horizontal and vertices strides, `strides = [1, stride, stride, 1]`.